In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib as mpl
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_regression
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from numpy.linalg import norm

In [ ]:
train = pd.read_csv("data/filled/train.csv")
test = pd.read_csv("data/filled/test.csv")

encoded_train = pd.read_csv("data/encoded/train.csv")
encoded_test = pd.read_csv("data/encoded/test.csv")

scaled_train = pd.read_csv("data/scaled/train.csv")
scaled_test = pd.read_csv("data/scaled/test.csv")

In [ ]:
RANDOM_STATE = 1

## EDA

In this notebook we will do some EDA with our data

<ol>
    <li><a href="#sale-price">Sale Price</a></li>
    <li><a href="#continuous">Continuous Features</a>
        <ul>
            <li><a href="#correlations">Correlations</a></li>
        </ul>
    </li>
    <li><a href="#pca-visualization">PCA analysis and visualization</a>
        <ul>
            <li><a href="#one-dimension">One Dimension</a></li>
            <li><a href="#two-dimensions">Two Dimensions</a></li>
            <li><a href="#three-dimensions">Three Dimensions</a></li>
            <li><a href="#most-affecting-pca">Checking the most affecting features in PCA</a></li>
            <li><a href="#comparing-pca-train-test">Comparing PCA visualizations for train and test</a></li>
            <li><a href="#pca-data-saving">Saving PCA data</a></li>
        </ul>
    </li>
    <li><a href="#anomaly-detection">Anomaly detection</a>
        <ul>
            <li><a href="#3-sigma-rule-pca">3 sigma rule for PCA</a></li>
            <li><a href="#isolation-forest">Isolation Forest</a></li>
        </ul>
    </li>
    <li><a href="#feature-selection">Feature Selection</a>
        <ul>
            <li><a href="#mutual-information">Mutual Information gain</a></li>
            <li><a href="#correlational-analysis">Correlational analysis</a></li>
            <li><a href="#random-forest-feature-importance">Random Forest feature importance</a></li>
        </ul>
    </li>
    <li><a href="#cluster-analysis">Cluster Analysis</a>
        <ul>
            <li><a href="#k-means-optimal-k">K-Means optimal number of clusters</a></li>
            <li><a href="#clustering-k-2">Clustering with K=2</a></li>
            <li><a href="#clustering-k-3">Clustering with K=3</a></li>
            <li><a href="#independent-cluster-pca">Independent Cluster PCA tranformation</a></li>
        </ul>
    </li>
</ol>

<a id="sale-price"></a>
## SalePrice

All begins with the target feature

In [ ]:
fig = px.histogram(train, x="SalePrice", title="SalePrice histogram")
fig.update_layout(width=1000, height=500, bargap=0.1)
fig.show()

We see that our target has long tail. BUT the tail is small in terms of object amount in it. There are only 9 objects with price which is higher than 500K. On this purpose we have already made a logarithmic transformation to our target during scaling

<a id="continuous"></a>
## Continuous features

In [ ]:
continuous_columns = train.select_dtypes(exclude="object").drop(["Id", "MSSubClass"], axis=1)
continuous_columns.columns.values

<a id="correlations"></a>
## Correlations

First of all let's look at the correlations of the continuous features including the target

In [ ]:
corr = continuous_columns.corr()

mask = np.triu(np.ones_like(corr, dtype=bool))

fig, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(230, 20, as_cmap=True)

sns.heatmap(corr, mask=mask, cmap=cmap)

Let's check the highest and the lowest correlations

In [ ]:
top_corr = corr.mask(mask, 0).unstack().sort_values(ascending=False)
print("TOP 10 HIGH CORRELATIONS\n", top_corr.head(10))
print("TOP 10 LOW CORRELATIONS\n", top_corr.tail(10))

We see that there are 6 pairs off features which have correlations greater than 0.7 and 2 pairs which have correlations lower than -0.4. This means that our dataset is high-correlated and models such simple linear regression can have some difficults during training on it

<a id="pca-visualization"></a>
## PCA analysis and visualization

Let's look at the first several components of PCA transformation of our data. We can check if our data is splitted into several easy distinguishable areas and how price of the houses are distributed among them.

Also we can check possible outliers and if the test data looks corresponding to the train one

In [ ]:
np.set_printoptions(suppress=True, precision=3)

N_components = 50

pca = PCA(n_components = N_components, random_state=RANDOM_STATE)

pca_train = pca.fit_transform(scaled_train.drop(["SalePrice"], axis=1))

print("WHOLE VARIANCE", scaled_train.drop(["SalePrice"], axis=1).var().sum())
print("EXPLAINED VARIANCE", pca.explained_variance_)
print("EXPLAINED VARIANCE RATIO", pca.explained_variance_ratio_)
print("FIRST 50 DIMENSION EXPLAINED VARIANCE", pca.explained_variance_ratio_.sum())

pca_train = pd.DataFrame(pca_train, columns=["pc{}".format(i) for i in range(1, 51)])
pca_train["SalePrice"] = scaled_train["SalePrice"]

We see that in our dataset there are no principal components which can describe the most part of the variance of the whole dataset since even the first 5 principal components describe only 37% of the variance.
The first 50 Principal components describe almost 88.54% of the whole variance

Anyway we can plot our first three "main" dimensions of the dataset

<a id="one-dimension"></a>
## One Dimension

In [ ]:
fig = px.scatter(pca_train, x="pc1",
                 y=[0 for i in range(pca_train.shape[0])],
                color="SalePrice")
fig.show()

Even for the first principal component we see that our sale price is increasing with increasing of the first components. In addition we see 2 values (on the right side) which obviously fall out of the whole dataset

<a id="two-dimensions"></a>
## Two Dimensions

In [ ]:
fig = px.scatter(pca_train, x="pc1", y="pc2", color="SalePrice")
fig.show()

That's interesting because we see that along the second principal component sale price almost doesn't change. It changes along the first component but almost the same along the second one

<a id="three-dimensions"></a>
## Three dimensions

In [ ]:
fig = px.scatter_3d(pca_train, x="pc1", y="pc2", z="pc3", color="SalePrice")
fig.update_traces(marker_size = 3)
fig.show()

Along the third component our sale price also doesn't change.

In addition it's very important to notice that our dataset seems to consist of two areas

Let's also plot dependence of SalePrice from pc1 and pc2 in 3D

In [ ]:
fig = px.scatter_3d(pca_train, x="pc1", y="pc2", z="SalePrice", color="SalePrice")
fig.update_traces(marker_size = 3)
fig.show()

Here we see almost clear linear dependency of SalePrice from pc1 and pc2

<a id="most-affecting-pca"></a>
## Checking the most affecting features in PCA

We can look which original features affect the most each of the first 10 principal component

In [ ]:
N = 5

principal_components = pca.components_

top_coeffs = np.sort(np.absolute(principal_components), axis=1)[:5, -N:]
top_features_idx = np.argsort(np.absolute(principal_components), axis=1)[:5, -N:]

print("TOP {} MOST AFFECTING FEATURES FOR EACH PC".format(N))
for ii, row in enumerate(top_features_idx):
    print("PC_{}".format(ii + 1))
    print("features:", np.flip(scaled_train.drop(["SalePrice"], axis=1).columns.values[row]))
    print("coefficients (abs):", np.flip(top_coeffs[ii]))

We see that the most valuable features for the first principal component are **OverallQual**, **GarageCars**, **GrLivArea**, **GarageArea** and **YearBuilt**

<a id="comparing-pca-train-test"></a>
## Comparing PCA visualizations for train and test

We can check if our scatter plots for train and test look similar after PCA transformation

In [ ]:
pca_test = pca.transform(scaled_test)

print("WHOLE VARIANCE", scaled_test.var().sum())

pca_test = pd.DataFrame(pca_test, columns=["pc{}".format(i) for i in range(1, 51)])

fig = px.scatter_3d(pca_test, x="pc1", y="pc2", z="pc3")
fig.update_traces(marker_size = 2)
fig.show()

We have pretty similar plot which means that train and test set has some similarity in their structure. That's pretty good because that means that trained on the train set models which perform well on it can perform also not bad on the test set (perhaps :)

## PCA data saving

Let's save our train and test dataset but with PCA transformation. We will use the first 50 principal components of our train dataset.

In [ ]:
pca_train.to_csv("data/pca/train50.csv", index=False)
pca_test.to_csv("data/pca/test50.csv", index=False)

<a id="anomaly-detection"></a>
## Anomaly detection

Now we will use some approaches for anomaly detection. Generally we can manually decide which train points seem to be anomaly based on the above PCA visualization but we will apply some general approaches.

## 3 Sigma rule on PCA

As the most easy step we can try to use simple 3 sigma rule on our PCA data

In [ ]:
sigma_train = pca_train.copy()

sigma_train["is_filtered"] = pd.Series(np.zeros(sigma_train.shape[0]), dtype="int64")

for column in sigma_train.columns.drop("SalePrice").values:
    
    values = sigma_train[column]
    mean = values.mean()
    std = values.std()
    
    sigma_train.loc[(mean - 3*std > values) | (values > mean + 3*std), "is_filtered"] = 1

print("CNT FILTERED OBJECTS", sigma_train["is_filtered"].sum())

fig = px.scatter_3d(pca_train, x="pc1", y="pc2", z="SalePrice", color=sigma_train["is_filtered"])
fig.update_traces(marker_size = 2)
fig.show()

3 sigma rule throws out to much objects

<a id="isolation-forest"></a>
## Isolation Forest

In [ ]:
isolated_train = pca_train.copy()

clf = IsolationForest(random_state=3, contamination=0.05).fit(isolated_train)
isolated_train["anomaly_score"] = clf.predict(isolated_train)

print("COUNT OF ANOMALIES:", isolated_train[isolated_train["anomaly_score"] == -1].shape[0])

fig = px.scatter_3d(pca_train, x="pc1", y="pc2", z="SalePrice", color=isolated_train["anomaly_score"])
fig.update_traces(marker_size = 2)
fig.show()

Isolation Forest throw out some objects which are too far from the main area of dataset.

<a id="feature-selection"></a>
## Feature selection

We've done feature extraction using the PCA and have noticed some interesting pattern that the first principal components mostly shows us changes in the target feature.

Let's use some feature selection techniques to see which features give us useful information and which do not.

We will not delete features in this notebook because I think that we have done enough manipulation to our data for the future modeling. We will check feature informativeness just to make some additional investigation

<a id="mutual-information"></a>
### Mutual Information gain

In [ ]:
mutual_train = encoded_train.drop(["SalePrice"], axis=1)
target = encoded_train["SalePrice"]

mis = mutual_info_regression(mutual_train, target)

N = 10


topNgain = np.flip(np.sort(mis)[-N:])
topNfeatures = np.flip(mutual_train.columns.values[np.argsort(mis)[-N:]])

botNfeatures = mutual_train.columns.values[np.argsort(mis)[0:N]]
botNgain = np.sort(mis)[0:N]

print("TOP MUTUAL GAIN FEATURES:", topNfeatures)
print("THEIR MUTUAL GAIN:", topNgain)
print("BOTTOM MUTUAL GAIN FEATURES", botNfeatures)
print("THEIR MUTUAL GAIN", botNgain)

In [ ]:
mis_series = pd.Series(mis, index=mutual_train.columns.values)

print("Count of features with mutual gain less than 1e-5:", mis_series[mis_series < 1e-5].shape)

The most interesting thing which we can notice is that in our top-10 mutual information gain features there are features which were also in the top-5 most affecting features for the first principal component

<a id="correlational-analysis"></a>
### Correlational analysis

Correlations analysis we have done above in the second stage of EDA. It can be used for understanding which features can be usefull for the linear models

<a id="random-forest-feature-importance"></a>
### Random Forest feature importance

In [ ]:
X = encoded_train.drop(["SalePrice"], axis=1)
y = encoded_train["SalePrice"]

forest = RandomForestRegressor(random_state=RANDOM_STATE)
forest.fit(X, y)
features_df = pd.DataFrame(forest.feature_importances_, columns=["importance"], index=X.columns)
features_df.sort_values(by="importance", ascending=False).head()

Random Forest makes almost all splits to increase informativity along the OverallQual component. We see that after the 2ndFlrSF feature other features are used less then in 3% cases.

In [ ]:
features_df[features_df["importance"] < 1e-3].shape

There are 278 features with importance less than 0.001

<a id="cluster-analysis"></a>
## Cluster Analysis

First of all. Let's look at our PCA visualization not through the first two principal components, but through the second and third.

In [ ]:
fig = px.scatter_3d(pca_train, x="pc2", y="pc3", z="SalePrice", color="SalePrice")
fig.update_traces(marker_size = 2)
fig.show()

What we see? We see that there are no so beautiful linear picture as we saw before. It looks like there are two areas. One locates higher with respect to Sale Price and another locates lower. 

Moreover if we take a look at the visualization using the 4th and 5th principal components we can also find out that our data is obviously splitted for two other areas. It is interesting for us that the second (smaller) area consists of low-priced houses

In [ ]:
fig = px.scatter_3d(pca_train, x="pc4", y="pc5", z="SalePrice", color="SalePrice")
fig.update_traces(marker_size = 2)
fig.show()

These thoughts give us a reason to make some clustering to our data to find some separation of the dataset into some areas which have different properties in terms of predicting the target

<a id="k-means-optimal-k"></a>
## K-Means optimal number of clusters

Let's use the widely known K-means algorithm and try to find optimal count of clusters in our data. First of all we will use the Elbow method.

In [ ]:
clustered_train = scaled_train.drop(["SalePrice"], axis=1).copy() 
root_of_sum_of_squared_distances = []
K = range(2,30)

for k in K:
    km = KMeans(n_clusters=k, random_state=RANDOM_STATE)
    km = km.fit(clustered_train)
    root_of_sum_of_squared_distances.append(km.inertia_**(1/2))

plt.plot(K, root_of_sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('root_of_sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

From the obtained plot it's not so obvious where is the optimal number of clusters. It seems the sum of squared distances is going down not smoothly. Let's calculate the silhouette score for our clustering

In [ ]:
silhouette_scores = []
for k in K:

    km = KMeans(n_clusters=k, random_state=2)
    cluster_labels = km.fit_predict(clustered_train)
    
    # Рассчитываем средний коэффициент силуэта для текущего K
    silhouette_avg = silhouette_score(clustered_train, cluster_labels)
    silhouette_scores.append(silhouette_avg)

plt.plot(K, silhouette_scores, 'bx-')
plt.xlabel('k')
plt.ylabel('silhouette_scores')
plt.title('Silhouette Method For Optimal k')
plt.show()

After K=3 our silhouette score strictly goes down. But generally even for K=2 and K=3 it's quite small.

<a id="clustering-k-2"></a>
## Clustering with K=2

Since all scores have values less than 0.12 and graph from elbow method goes down smoothly it seem's that K-means cannot give us some good clustering

Let's check how our data looks after the clastering with K=2

In [ ]:
km2 = KMeans(n_clusters=2, random_state=2)
km2_labels = km2.fit_predict(clustered_train)

In [ ]:
fig = px.scatter_3d(pca_train, x="pc1", y="pc2", z="pc3", color=km2_labels)
fig.update_traces(marker_size = 2)
fig.show()

With K=2 the K-means algorithm splits our dataset into two clasters with different kind of sale prices. We have noticed these arease before when we made the first plot of our data in PCA coordinates
The blue area has higher sale prices than the yellow one.

It means that there is some natural metric difference between objects with high price and low price. Probably we can use this information in futhure modeling.

<a id="clustering-k-3"></a>
## Clustering with K=3

In [ ]:
km3 = KMeans(n_clusters=3, random_state=2)
km3_labels = km3.fit_predict(clustered_train)

fig = px.scatter_3d(pca_train, x="pc2", y="pc3", z="SalePrice", color=km3_labels)
fig.update_traces(marker_size = 2)
fig.show()

With respect to pc2 and pc3 we cannot unterstand why k-means gives us exactly this kind of cluster splitting. But if we look at our data from the side of pc4 and pc5, than

In [ ]:
fig = px.scatter_3d(pca_train, x="pc4", y="pc5", z="SalePrice", color=km3_labels)
fig.update_traces(marker_size = 2)
fig.show()

We see one more natural split in our data. One cluster is for high-valued houses. Two others are with low-valued houses but as we see the low-values houses seem also to be splitted into two different groups. 

Because of the fact that we can split "blue" group of the other two by just the coordinate pc3 that means that for example desicion tree can see this kind of splitting. Our second split between high-valued houses and low-valued houses also can be spoted by simple Desicion Tree or Random Forest because we saw that SalePrice are highly changing along the pc1 coordinate which can be used by Decision Tree.

But these are only reasonings for the Decision Trees. It also can be that in different clusters the data has different behaviour and can be modeled with different models

<a id="independent-cluster-pca"></a>
## Independent Cluster PCA tranformation

Let's try to do one more thing. Perhaps it makes sense to cluster our dataset and only them use PCA independently to each cluster.
The reason for this can be that there can be a situation when we will receive much difference in Principal components in each cluster.
This means that data in each cluster has some independent patterns from the point of PCA view.

We will do our visualization for K=3

In [ ]:
clusters = pd.Series(km3_labels)

scaler = StandardScaler()

cluster1 = scaler.fit_transform(encoded_train[clusters == 0].drop("SalePrice", axis=1))
cluster2 = scaler.fit_transform(encoded_train[clusters == 1].drop("SalePrice", axis=1))
cluster3 = scaler.fit_transform(encoded_train[clusters == 2].drop("SalePrice", axis=1))

pca1 = PCA(n_components = 50, random_state=RANDOM_STATE)
pca2 = PCA(n_components = 50, random_state=RANDOM_STATE)
pca3 = PCA(n_components = 50, random_state=RANDOM_STATE)

pca_cluster1 = pca1.fit_transform(cluster1)
pca_cluster2 = pca2.fit_transform(cluster2)
pca_cluster3 = pca3.fit_transform(cluster3)

pca_cluster1 = pd.DataFrame(pca_cluster1, columns=["pc{}".format(i) for i in range(1, 51)])
pca_cluster2 = pd.DataFrame(pca_cluster2, columns=["pc{}".format(i) for i in range(1, 51)])
pca_cluster3 = pd.DataFrame(pca_cluster3, columns=["pc{}".format(i) for i in range(1, 51)])

pca_cluster1["SalePrice"] = scaled_train[clusters == 0]["SalePrice"].values
pca_cluster2["SalePrice"] = scaled_train[clusters == 1]["SalePrice"].values
pca_cluster3["SalePrice"] = scaled_train[clusters == 2]["SalePrice"].values

## Explained variance in each cluster after PCA

In [ ]:
print("EXPLAINED VARIANCE RATIO CLUSTER 1", pca1.explained_variance_ratio_[:5])
print("EXPLAINED VARIANCE RATIO CLUSTER 2", pca2.explained_variance_ratio_[:5])
print("EXPLAINED VARIANCE RATIO CLUSTER 3", pca3.explained_variance_ratio_[:5])

## Distance between first principle components

In [ ]:
print("PC1 distance between cluster 1 and 2:", norm(pca1.components_[0] - pca2.components_[0]))
print("PC1 distance between cluster 2 and 3:", norm(pca2.components_[0] - pca3.components_[0]))
print("PC1 distance between cluster 3 and 4:", norm(pca3.components_[0] - pca1.components_[0]))

In [ ]:
fig = px.scatter_3d(pca_cluster1, x="pc1", y="pc2", z="SalePrice", color="SalePrice")
fig.update_traces(marker_size = 2)
fig.show()

In [ ]:
fig = px.scatter_3d(pca_cluster2, x="pc1", y="pc2", z="SalePrice", color="SalePrice")
fig.update_traces(marker_size = 2)
fig.show()

In [ ]:
fig = px.scatter_3d(pca_cluster3, x="pc1", y="pc2", z="SalePrice", color="SalePrice")
fig.update_traces(marker_size = 2)
fig.show()

For now I don't see some obvious application of such a clustering and transformation. It's hard to say if some well-known model can perform good on this data. 

We must spend more time for cluster analysis because we have used only one algorithm. Moreover K-means can give us different results based on the random_state parameter. Perhaps there are some more cool clustering but we didn't get it

This EDA file is large and I prefer to continue cluster analysis in another new notebook

The first EDA file is finished...